In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
import glob

# 멕북용 폰트 불러오기 무시해주세용
font_path = '/System/Library/Fonts/Supplemental/AppleGothic.ttf'
font = fm.FontProperties(fname=font_path).get_name()
matplotlib.rc('font', family=font)

In [3]:
# 푸드 데이터 로드
path_food = "./data/EC_SNS_KFOOD_ATTRACTION_DATA_*.csv"
files_food = glob.glob(path_food)
df_food = [pd.read_csv(f, encoding='utf-8') for f in files_food]
sns_kfood = pd.concat(df_food, ignore_index=True)

# 뷰티 데이터 로드
path_beauty = "./data/EC_SNS_KBEAUTY_ATTRACTION_DATA_*.csv"
files_beauty = glob.glob(path_beauty)
df_beauty = [pd.read_csv(f, encoding='utf-8') for f in files_beauty]
sns_beauty = pd.concat(df_beauty, ignore_index=True)

# 데이터 병합
full_df = pd.concat([sns_kfood, sns_beauty], ignore_index=True)

full_df.to_csv('full_df.csv',index=False)

### 필요한 컬럼명 정리
#### 그대로 readme에 올려도 될듯합니다
|컬럼영문명|컬럼한글명|필요 컬럼|
|---|---|---|
|TRRSRT_NM|관광지명||
|BASE_YM|기준년월|O|
|CHNNEL_NM|채널명||
|BASE_YEAR_ACCMLT_FQ_CO|기준년도누적빈도수||
|BASE_YM_FQ_CO|	기준년월빈도수||
|BASE_YEAR_BEFORE_MT_FQ_CO|기준년도이전월빈도수||
|BEFORE_MT_VERSUS_FQ_CO_IRDS_RT|이전월대비빈도수증감율||
|AVRG_SCORE_VALUE|평점값|O|
|REVIEW_CO|리뷰수|O|
|PLACE_TY|장소유형||
|LC_LA|위치위도||
|LC_LO|위치경도||
|ADDR|주소||
|CTPRVN_NM|시도명|O|
|SIGNGU_NM|시군구명|O|
|TURSM_CSTMR_CO|관광고객수|O|
|BEFORE_YEAR_MT_TURSM_CSTMR_CO|이전년도월관광고객수||
|TURSM_CSTMR_CO_IRDS_RT|관광고객수증감율||
|TURSM_SPND_PRICE|관광소비금액|O|
|NATIVE_TURSM_SPND_PRICE|내국인관광소비금액|O|
|FRNR_TURSM_SPND_PRICE|외국인관광소비금액|O|

In [4]:
# 채널 이름이 체널 전체인 것들만 추출 후 0으로 초기화
full_df = full_df[full_df['CHNNEL_NM'] == '채널전체'].copy()
full_df = full_df.fillna(0)


# 사용할 컬럼 추출
use_cols = ['BASE_YM', 'CTPRVN_NM', 'SIGNGU_NM', 'TURSM_CSTMR_CO', 
            'TURSM_SPND_PRICE', 'NATIVE_TURSM_SPND_PRICE', 
            'FRNR_TURSM_SPND_PRICE', 'AVRG_SCORE_VALUE', 'REVIEW_CO', 
            'BASE_YM_FQ_CO', 'BASE_YEAR_BEFORE_MT_FQ_CO']
df_sub = full_df[use_cols].copy()

# 데이터 형 변환
df_sub['BASE_YM'] = pd.to_datetime(df_sub['BASE_YM'], format='%Y%m')
df_sub = df_sub.fillna(0)

# 데이터 값 변환 (인천-> 인천광역시, 서울-> 서울특별시 등)
df_sub['SIGNGU_NM'] = df_sub['SIGNGU_NM'].replace({
    '인천': '인천광역시', 
    '서울':'서울특별시',
    '서울시': '서울특별시',
    '강원특별자치도': '강원도',
    '고양시':'경기도',
    '남양주시':'경기도',
    '부산':'부산광역시',
    '성남시':'경기도',
    '수원시': '경기도',
    '인천':'인천광역시',
    '전북특별자치도':'전라북도',
    '천안시':'강원도'
    })

# 통합 데이터 셋 (뷰티는 구별로 다른 구간도 있고 같은 구간도 있어서 평균으로 처리함)    
final_df = df_sub.groupby(['BASE_YM', 'CTPRVN_NM', 'SIGNGU_NM']).agg({
    'TURSM_SPND_PRICE': 'mean',          # 관광소비금액
    'TURSM_CSTMR_CO': 'mean',            # 관광고객수
    'NATIVE_TURSM_SPND_PRICE': 'mean',   # 내국인관광소비금액
    'FRNR_TURSM_SPND_PRICE': 'mean',     # 외국인관광소비금액
    'AVRG_SCORE_VALUE': 'mean',          # 평점값
    'REVIEW_CO': 'mean',                 # 리뷰수
    'BASE_YM_FQ_CO': 'sum',              # 기준년월빈도수
    'BASE_YEAR_BEFORE_MT_FQ_CO' : 'sum'  # SNS기준년도이전우러빈도수
}).reset_index()

# SNS 한 달 간의 증감량
final_df['SNS_MONTHLY_DIFF'] = final_df['BASE_YM_FQ_CO'] - final_df['BASE_YEAR_BEFORE_MT_FQ_CO']


In [5]:

# pct_change(): 전년대비 변화량을 계산해 주는 함수 <(현재값 - 이전값) / 이전값>
# diff() : 이전값 대비 현재값의 변화량을 계산해 주는 함수 <현재값 - 이전값>
# (현재값 - 이전값) / 이전값 * 100 = 현재 성장률
final_df = final_df.sort_values(['SIGNGU_NM', 'BASE_YM'])   # pct_change()를 쓰려면 정렬이 필요
final_df = final_df.reset_index(drop=True)

# 몇몇개의 시군구명이 KR로 들어가 있는 데이터랑 '0' 들어가 있는 데이터 제거
final_df = final_df[~final_df['SIGNGU_NM'].isin(['KR', '0', 0])]
final_df = final_df.reset_index(drop=True)

# 전월 대비 소비 증감률 = (현재달 - 지난달) / 지난달 * 100
# final_df['SPND_GROWTH_RATE'] = final_df.groupby('SIGNGU_NM')['TURSM_SPND_PRICE'].pct_change() * 100
# 전월 대비 소비 증감액 = 현재달 - 지난달 (단위: 백만원)
final_df['SPND_DIFF_VALUE'] = final_df.groupby('SIGNGU_NM')['TURSM_SPND_PRICE'].diff() / 1000000

# 관광고객수 증감률 / 증감액
# final_df['CSTMR_GROWTH_RATE'] = final_df.groupby('SIGNGU_NM')['TURSM_CSTMR_CO'].pct_change() * 100
final_df['CSTMR_DIFF_VALUE'] = final_df.groupby('SIGNGU_NM')['TURSM_CSTMR_CO'].diff()

# 리뷰수 증감률 / 증감수
# final_df['REVIEW_GROWTH_RATE'] = final_df.groupby('SIGNGU_NM')['REVIEW_CO'].pct_change() * 100
final_df['REVIEW_DIFF_VALUE'] = final_df.groupby('SIGNGU_NM')['REVIEW_CO'].diff()

# 검색빈도수 증감수
final_df['SNS_DIFF_VALUE'] = final_df.groupby('SIGNGU_NM')['BASE_YEAR_BEFORE_MT_FQ_CO'].diff()   

# 결측치 및 무한대를 이전 값으로 처리 
# 변화률 위주이기 때문에 0을 넣어서 이전달과 변화가 없다 라고 평가하는게 맞다고 봄
final_df = final_df.replace([np.inf, -np.inf], np.nan)
final_df = final_df.fillna(0)

# 이상치 제거
# 중간에 보면 증감률이 8000 까지 뛸때가 있음
# 실측 오류이거나 데이터 등록에서 오류가 발생했다고 보고 마찬가지로 변화가 없다는 의미의 0으로 처리
# final_df.loc[abs(final_df['SPND_GROWTH_RATE']) > 500, ['SPND_GROWTH_RATE', 'SPND_DIFF_VAL_M']] = 0
# final_df.loc[abs(final_df['CSTMR_GROWTH_RATE']) > 500, ['CSTMR_GROWTH_RATE', 'CSTMR_DIFF_VALUE']] = 0
# final_df.loc[abs(final_df['REVIEW_GROWTH_RATE']) > 500, ['REVIEW_GROWTH_RATE', 'REVIEW_DIFF_VALUE']] = 0

# 증감률 즉 퍼센테이지는 당장에 필요 없을꺼같아서 우선 주석처리 해봤습니다.

final_df

,BASE_YM,CTPRVN_NM,SIGNGU_NM,TURSM_SPND_PRICE,TURSM_CSTMR_CO,NATIVE_TURSM_SPND_PRICE,FRNR_TURSM_SPND_PRICE,AVRG_SCORE_VALUE,REVIEW_CO,BASE_YM_FQ_CO,BASE_YEAR_BEFORE_MT_FQ_CO,SNS_MONTHLY_DIFF,SPND_DIFF_VALUE,CSTMR_DIFF_VALUE,REVIEW_DIFF_VALUE,SNS_DIFF_VALUE
0,2025-02-01,경기도,가평군,4526283.0,1330686.0,4356036.0,170247.0,4.4,36432.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
1,2025-07-01,경기도,가평군,655573.0,2688308.0,285133.0,370440.0,4.4,36800.000000,1.0,0.0,1.0,-3.870710,1357622.0,368.000000,0.0
2,2025-08-01,경기도,가평군,56729443.0,3322011.0,56258056.0,471387.0,4.4,36877.000000,1.0,1.0,0.0,56.073870,633703.0,77.000000,1.0
3,2025-09-01,경기도,가평군,39090303.0,2051913.0,38434403.0,655900.0,4.4,30306.666667,2.0,1.0,1.0,-17.639140,-1270098.0,-6570.333333,0.0
4,2025-10-01,경기도,가평군,48143693.0,2644120.0,47364911.0,778782.0,4.4,30416.666667,0.0,2.0,-2.0,9.053390,592207.0,110.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,2025-11-01,강원특별자치도,홍천군,33648648.0,1640793.0,33445264.0,203384.0,4.3,6697.000000,0.0,0.0,0.0,-9.659481,-606483.0,2.000000,0.0
837,2025-12-01,강원특별자치도,홍천군,36272116.0,1632712.0,34719603.0,1552513.0,4.3,6705.000000,0.0,0.0,0.0,2.623468,-8081.0,8.000000,0.0
838,2025-03-01,서울특별시,화양동,37341094.0,6797980.0,22373501.0,14967593.0,4.1,812.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
839,2025-04-01,서울특별시,화양동,33977729.0,5887682.0,21736852.0,12240877.0,4.1,817.000000,0.0,0.0,0.0,-3.363365,-910298.0,5.000000,0.0


## ML